### 参数管理

#### 首先关心单隐藏层 的多层感知机

In [1]:
import torch
from torch import nn

# 线性层 relu 线性层 =》 单隐藏层的MLP
# 输出层 8 * 1
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))

net(X)

tensor([[ 0.1766],
        [-0.0226]], grad_fn=<AddmmBackward0>)

### 参数管理

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0782,  0.1601, -0.3468, -0.0973, -0.1932,  0.1326, -0.0241,  0.2809]])), ('bias', tensor([0.3106]))])


### 目标参数

In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.3106], requires_grad=True)
tensor([0.3106])


In [4]:
# 访问每个参数的梯度
# 由于还没有调用反向传播，所以参数的梯度处于初始状态
net[2].weight.grad == None

True

### 一次性访问所有参数

In [8]:
# 递归整个树来提取每个子块的参数

# 访问第一个全连接层
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
# 访问所有层
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [9]:
# 另一种访问网络参数方式
net.state_dict()['2.bias'].data

tensor([0.3106])

### 参数初始化

### 内置初始化

In [11]:
# 正态初始化 m 是 nn.Module
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01) # 均值为0，方差为0.01
        nn.init.zeros_(m.bias)

# 遍历整个网络 初始化所有参数
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0022, -0.0099,  0.0271, -0.0056]), tensor(0.))

In [12]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

### 对某些块应用不同的初始化方法

In [13]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.5458,  0.5677,  0.3297,  0.2122])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化

In [14]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 9.0449, -8.6874,  0.0000,  0.0000],
        [-9.2371, -0.0000, -5.3518,  0.0000]], grad_fn=<SliceBackward0>)

 ### 参数绑定

In [15]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
